In [1]:
import tensorflow as tf
import numpy as np

In [11]:
# List out our bandits.
bandits = [0.2, 0, -0.2, -5]
num_bandits = len(bandits)

# define reward
def pullBandit(bandit):
    result = np.random.randn(1)
    if result > bandit:
        return 1
    else:
        return -1

In [18]:
tf.reset_default_graph()

# Use neural network to approximate the policy
weight = tf.Variable(tf.ones(num_bandits))
chosen_action = tf.arg_max(weight, 0)

# Establish the training procedure
# We feed the reward and chosen action into the network
reward_holder = tf.placeholder(shape = [1], dtype=tf.float32)
action_holder = tf.placeholder(shape = [1], dtype=tf.int32)
responsible_weight = tf.slice(weight, action_holder, [1])
loss = -(responsible_weight*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [19]:
total_episodes = 1000
total_reward = np.zeros(num_bandits)
e = 0.3 # the chance of taking a random action

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(total_episodes):
        # choose either a random action or one from network
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        reward = pullBandit(bandits[action])
        
        # Update the network
        _, resp, ww = sess.run([update, responsible_weight, weight],
                               feed_dict = {reward_holder: [reward], action_holder:[action]})
        
        # Update our running tally of scores
        total_reward[action] += reward
        
        if i&50 == 0:
            print("Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))

print("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 bandits: [-1.  0.  0.  0.]
Running reward for the 4 bandits: [-1.  0.  0.  1.]
Running reward for the 4 bandits: [-1.  0.  1.  3.]
Running reward for the 4 bandits: [-1.  0.  1.  4.]
Running reward for the 4 bandits: [-1.  0.  0.  6.]
Running reward for the 4 bandits: [-1.  0.  0.  7.]
Running reward for the 4 bandits: [-2.  0.  0.  9.]
Running reward for the 4 bandits: [ -2.   0.   0.  10.]
Running reward for the 4 bandits: [ -5.   0.  -1.  51.]
Running reward for the 4 bandits: [ -5.   0.  -1.  52.]
Running reward for the 4 bandits: [ -5.   0.  -1.  55.]
Running reward for the 4 bandits: [ -5.   0.  -1.  56.]
Running reward for the 4 bandits: [ -5.   0.  -1.  59.]
Running reward for the 4 bandits: [ -5.   0.  -1.  60.]
Running reward for the 4 bandits: [ -5.   0.  -1.  63.]
Running reward for the 4 bandits: [ -5.   0.  -1.  64.]
Running reward for the 4 bandits: [  -6.    0.    0.  107.]
Running reward for the 4 bandits: [  -6.    0.    0.  108.]
Running rewa